<a href="https://colab.research.google.com/github/SvA1/Compling/blob/master/HW_3(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt

In [0]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [0]:
import re
import gzip, csv
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)
from collections import Counter, defaultdict
from pprint import pprint
from nltk import sent_tokenize

!pip install symspellpy
from symspellpy import SymSpell, Verbosity
import pkg_resources

In [0]:
corpus = open('corpus_5000.txt', 'w')
with gzip.open('lenta-ru-news.csv.gz', 'rt') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i < 5000: 
            corpus.write(line[2].replace('\xa0', ' ') + '\n')

In [0]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
import nltk
nltk.download('punkt')
corpus = []
for text in open('corpus_5000.txt').read().splitlines()[:12000]:
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
corpus_path = 'corpus_5000.txt'
dictionary_path = sym_spell.create_dictionary(corpus_path)

In [0]:
def Sym_spell_func(input_term):
  sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
  suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
                               max_edit_distance=2, include_unknown=True)
  return str(suggestions[0]).split(',')[0]
Sym_spell_func('ооочень')

'очень'

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
print(bad[1])
print(true[1])

Опофеозом дня для меня сегодня стала фраза услышанная в новостях:
Апофеозом дня для меня сегодня стала фраза услышанная в новостях


In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [0]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


In [0]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7020979020979021
0.5003837298541827
0.26771563110141267


In [0]:
def deletes(word):
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])     for i in range(len(word) + 1)]
    deletes1    = [L + R[1:]               for L, R in splits if R]
    splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
    deletes2    = [L + R[2:]               for L, R in splits2 if R]
    deletes = deletes1 + deletes2
    return set(deletes)
    print("\n")

In [0]:
corpus_sents = open('corpus_5000.txt', encoding='utf8').read().lower().split()
true_list = [re.sub('(^\W+|\W+$)', '', token) for token in corpus_sents if (set(token)-punct)]

true_dict = {}
prob_dict = {}
for i in true_list:
  prob_dict[i] = prob_dict.get(i, 0) + 1
  for j in deletes(i):
    true_dict[j] =  i
print(list(true_dict.keys())[:5])

['txt', 'tet', 'xt', 'tex', 'tt']


In [0]:
def prob(word):
  p = prob_dict.get(word, 0)/len(set(true_list))
  return(p)

In [0]:
def in_pro(input_word):
  candidates_list = []
  local = {}
  
  if input_word in prob_dict:
    return input_word
  if input_word in true_dict:
    candidates_list.append(true_dict[input_word])
  for j in deletes(input_word):
    if j in true_dict:
        candidates_list.append(true_dict[j])
        candidates_list = list(set(candidates_list))
  if len(candidates_list) > 1:
    for i in set(candidates_list):
      local[prob(i)] = i
      sorted_probs = sorted(list(local.keys()))
      return local[sorted_probs[0]]
  else:
    if len(candidates_list) > 0:  
     return candidates_list[0]
    else:
      return input_word + ""

In [0]:
print(in_pro('опофеозом'))
print(in_pro('надаело'))
print(in_pro('оочень'))

апофеозом
надоело
огонь


In [0]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7020979020979021
0.5003837298541827
0.26771563110141267


### Триграммная модель

In [0]:
corpus_news = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]

In [0]:
corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [0]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_news:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, n=3))

In [0]:
id2word_unigrams = list(unigrams)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}

id2word_bigrams = list(bigrams)
word2id_bigrams = {word:i for i, word in enumerate(id2word_bigrams)}

In [0]:
def get_closest_match_vec(text, X, vec, topn=20):
    # превращаем слово в вектор такой же размерности
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0] #distance - чем больше, тем хуже, а similarity наоборот
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [0]:
def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    # ваш код здесь
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    sims = Counter()
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup,topn, metric=metric)
    
    return closest

get_closest_hybrid_match('сонце', X, vec)